## WORKSHOP 1: LIMPIEZA DE DATOS

<a id="section_toc"></a> 
## ÍNDICE

[Intro](#section_intro)

$\hspace{.5cm}$[Dataset: Primeras Aproximaciones](#section_dataset)

[Imputación de datos faltantas](#section_imputacion)

$\hspace{.5cm}$[Geopandas](#section_geopandas)

$\hspace{1cm}$[Regex: Countries](#section_countries)

$\hspace{1cm}$[Regex: Amenities](#section_amenities)

$\hspace{.5cm}$[Surface](#section_surface)

$\hspace{.5cm}$[Regex: Rooms](#section_regex)

$\hspace{.5cm}$[Filtrado](#section_filtrado)

$\hspace{1cm}$[Criterio de filtrado: Price_aprox_usd](#section_price)

$\hspace{1cm}$[Filtrado Ciudad](#section_ciudad)

[Estimación Precio x m2, en función de los datos limpios](#section_price_m2)


<a id="section_intro"></a> 
## Intro

[volver a TOC](#section_toc)

En primer instancia, se cargan las bibliotecas a utilizar. Luego se cargará el dataset y se realizará una primer análisis exploratorio observando las columnas, y el porcentaje de datos faltantes

In [83]:
import pandas as pd
import numpy as np
import geopandas as gpd
import re


import shapely # genera las figuras geometricas
import descartes # relaciona shapely con matplotlib
import pyproj # proyecciones. Transformar coordenadas

In [84]:
data = pd.read_csv("properatti.csv")
print(data.shape)
data.head()

(121220, 26)


,Unnamed: 0,operation,property_type,place_name,place_with_parent_names,country_name,state_name,geonames_id,lat-lon,lat,...,surface_covered_in_m2,price_usd_per_m2,price_per_m2,floor,rooms,expenses,properati_url,description,title,image_thumbnail
0,0,sell,PH,Mataderos,|Argentina|Capital Federal|Mataderos|,Argentina,Capital Federal,3430787.0,"-34.6618237,-58.5088387",-34.661824,...,40.0,1127.272727,1550.000000,NaN,NaN,NaN,http://www.properati.com.ar/15bo8_venta_ph_mat...,"2 AMBIENTES TIPO CASA PLANTA BAJA POR PASILLO,...",2 AMB TIPO CASA SIN EXPENSAS EN PB,https://thumbs4.properati.com/8/BluUYiHJLhgIIK...
1,1,sell,apartment,La Plata,|Argentina|Bs.As. G.B.A. Zona Sur|La Plata|,Argentina,Bs.As. G.B.A. Zona Sur,3432039.0,"-34.9038831,-57.9643295",-34.903883,...,NaN,NaN,NaN,NaN,NaN,NaN,http://www.properati.com.ar/15bob_venta_depart...,Venta de departamento en décimo piso al frente...,VENTA Depto 2 dorm. a estrenar 7 e/ 36 y 37 ...,https://thumbs4.properati.com/7/ikpVBu2ztHA7jv...
2,2,sell,apartment,Mataderos,|Argentina|Capital Federal|Mataderos|,Argentina,Capital Federal,3430787.0,"-34.6522615,-58.5229825",-34.652262,...,55.0,1309.090909,1309.090909,NaN,NaN,NaN,http://www.properati.com.ar/15bod_venta_depart...,2 AMBIENTES 3ER PISO LATERAL LIVING COMEDOR AM...,2 AMB 3ER PISO CON ASCENSOR APTO CREDITO,https://thumbs4.properati.com/5/SXKr34F_IwG3W_...
3,3,sell,PH,Liniers,|Argentina|Capital Federal|Liniers|,Argentina,Capital Federal,3431333.0,"-34.6477969,-58.5164244",-34.647797,...,NaN,NaN,NaN,NaN,NaN,NaN,http://www.properati.com.ar/15boh_venta_ph_lin...,PH 3 ambientes con patio. Hay 3 deptos en lote...,PH 3 amb. cfte. reciclado,https://thumbs4.properati.com/3/DgIfX-85Mog5SP...
4,4,sell,apartment,Centro,|Argentina|Buenos Aires Costa Atlántica|Mar de...,Argentina,Buenos Aires Costa Atlántica,3435548.0,"-38.0026256,-57.5494468",-38.002626,...,35.0,1828.571429,1828.571429,NaN,NaN,NaN,http://www.properati.com.ar/15bok_venta_depart...,DEPARTAMENTO CON FANTÁSTICA ILUMINACIÓN NATURA...,DEPTO 2 AMB AL CONTRAFRENTE ZONA CENTRO/PLAZA ...,https://thumbs4.properati.com/5/xrRqlNcSI_vs-f...


<a id="section_dataset"></a> 
### Dataset: Primeras Aproximaciones

[volver a TOC](#section_toc)

Se busca obtener el porcentaje de nulos de cada columna con el fin de tomar decisiones sobre las mismas

In [85]:
data_null = data.isnull().sum()
data_null_porc = data_null.apply(lambda x: (x/data.shape[0])*100)
data_null_porc

Unnamed: 0                     0.000000
operation                      0.000000
property_type                  0.000000
place_name                     0.018974
place_with_parent_names        0.000000
country_name                   0.000000
state_name                     0.000000
geonames_id                   15.440521
lat-lon                       42.525986
lat                           42.525986
lon                           42.525986
price                         16.837156
currency                      16.837981
price_aprox_local_currency    16.837156
price_aprox_usd               16.837156
surface_total_in_m2           32.443491
surface_covered_in_m2         16.422208
price_usd_per_m2              43.394654
price_per_m2                  27.686850
floor                         93.483749
rooms                         60.905791
expenses                      88.234615
properati_url                  0.000000
description                    0.001650
title                          0.000000


Se puede ver que las columnas expenses y floor poseen un porcentaje de nulos muy elevados, los cuales se podrían bajar utilizando la columna description. Sin embargo, muy probablemente el porcentaje de estos campos no baje muy significativamente, por lo que se pueden tomar 2 posibles decisiones: eliminar las columnas, o simplemente dejarlas a modo informativo.
Se elije la 2da opción.

En función a los valores observados, se procede a imputar los datos faltantes respecto a las columnas de geolocalización.

<a id="section_imputacion"></a> 

## Imputación de datos faltantes

[volver a TOC](#section_toc)

<a id="section_geopandas"></a> 

### Geopandas

[volver a TOC](#section_toc)

Lo primero que se realiza es separar los datos de la columna "place_with_parent_names" para entender qué información posee dicha columna y poder obtener el dato de la ciudad, ya que dicho dato falta en el set.

In [86]:
splited = data.loc[:,"place_with_parent_names"].str.split(pat="|",expand=True)
splited = splited.rename(columns={1:"País",2:"Provincia",3:"Ciudad",4:"Barrio/Zona/Country",5:"Country/Barrio_Cerrado"})
data_split = pd.merge(data,splited,on=data.index)
data_split = data_split.drop(["key_0","País",0,6],axis=1)
data_split.head(5)

,Unnamed: 0,operation,property_type,place_name,place_with_parent_names,country_name,state_name,geonames_id,lat-lon,lat,...,rooms,expenses,properati_url,description,title,image_thumbnail,Provincia,Ciudad,Barrio/Zona/Country,Country/Barrio_Cerrado
0,0,sell,PH,Mataderos,|Argentina|Capital Federal|Mataderos|,Argentina,Capital Federal,3430787.0,"-34.6618237,-58.5088387",-34.661824,...,NaN,NaN,http://www.properati.com.ar/15bo8_venta_ph_mat...,"2 AMBIENTES TIPO CASA PLANTA BAJA POR PASILLO,...",2 AMB TIPO CASA SIN EXPENSAS EN PB,https://thumbs4.properati.com/8/BluUYiHJLhgIIK...,Capital Federal,Mataderos,,None
1,1,sell,apartment,La Plata,|Argentina|Bs.As. G.B.A. Zona Sur|La Plata|,Argentina,Bs.As. G.B.A. Zona Sur,3432039.0,"-34.9038831,-57.9643295",-34.903883,...,NaN,NaN,http://www.properati.com.ar/15bob_venta_depart...,Venta de departamento en décimo piso al frente...,VENTA Depto 2 dorm. a estrenar 7 e/ 36 y 37 ...,https://thumbs4.properati.com/7/ikpVBu2ztHA7jv...,Bs.As. G.B.A. Zona Sur,La Plata,,None
2,2,sell,apartment,Mataderos,|Argentina|Capital Federal|Mataderos|,Argentina,Capital Federal,3430787.0,"-34.6522615,-58.5229825",-34.652262,...,NaN,NaN,http://www.properati.com.ar/15bod_venta_depart...,2 AMBIENTES 3ER PISO LATERAL LIVING COMEDOR AM...,2 AMB 3ER PISO CON ASCENSOR APTO CREDITO,https://thumbs4.properati.com/5/SXKr34F_IwG3W_...,Capital Federal,Mataderos,,None
3,3,sell,PH,Liniers,|Argentina|Capital Federal|Liniers|,Argentina,Capital Federal,3431333.0,"-34.6477969,-58.5164244",-34.647797,...,NaN,NaN,http://www.properati.com.ar/15boh_venta_ph_lin...,PH 3 ambientes con patio. Hay 3 deptos en lote...,PH 3 amb. cfte. reciclado,https://thumbs4.properati.com/3/DgIfX-85Mog5SP...,Capital Federal,Liniers,,None
4,4,sell,apartment,Centro,|Argentina|Buenos Aires Costa Atlántica|Mar de...,Argentina,Buenos Aires Costa Atlántica,3435548.0,"-38.0026256,-57.5494468",-38.002626,...,NaN,NaN,http://www.properati.com.ar/15bok_venta_depart...,DEPARTAMENTO CON FANTÁSTICA ILUMINACIÓN NATURA...,DEPTO 2 AMB AL CONTRAFRENTE ZONA CENTRO/PLAZA ...,https://thumbs4.properati.com/5/xrRqlNcSI_vs-f...,Buenos Aires Costa Atlántica,Mar del Plata,Centro,


Luego, se observa la cantidad de nulos que posee el campo Ciudad.

In [87]:
data_split.loc[data_split.Ciudad=="",:].Provincia.value_counts()

Córdoba                         2648
Capital Federal                 1297
Bs.As. G.B.A. Zona Norte         222
Mendoza                          130
Buenos Aires Interior            106
Tucumán                           77
Bs.As. G.B.A. Zona Oeste          65
Misiones                          44
Santa Fe                          33
Buenos Aires Costa Atlántica      27
San Luis                          24
Bs.As. G.B.A. Zona Sur            24
Salta                             21
Chubut                            17
Neuquén                           10
Río Negro                          8
San Juan                           7
Tierra Del Fuego                   6
Catamarca                          3
Santiago Del Estero                3
La Pampa                           3
Corrientes                         2
Jujuy                              1
Chaco                              1
La Rioja                           1
Name: Provincia, dtype: int64

Se transforma data_split en geoDataFrame

In [88]:
geo_data = gpd.GeoDataFrame(data_split, geometry=gpd.points_from_xy(data_split.lon, data_split.lat))
geo_data.sample(2)

,Unnamed: 0,operation,property_type,place_name,place_with_parent_names,country_name,state_name,geonames_id,lat-lon,lat,...,expenses,properati_url,description,title,image_thumbnail,Provincia,Ciudad,Barrio/Zona/Country,Country/Barrio_Cerrado,geometry
85910,85910,sell,apartment,Rosario,|Argentina|Santa Fe|Rosario|,Argentina,Santa Fe,3838574.0,NaN,NaN,...,NaN,http://www.properati.com.ar/1b3om_venta_depart...,Sergio Vilella Inmobiliaria presenta departame...,"Condominios del Alto 4 - Dos dormitorios, cali...",https://thumbs4.properati.com/6/tiASAS8uc_cHdU...,Santa Fe,Rosario,,None,POINT (nan nan)
47202,47202,sell,apartment,Mar del Plata,|Argentina|Buenos Aires Costa Atlántica|Mar de...,Argentina,Buenos Aires Costa Atlántica,3430863.0,"-38.0051554,-57.5444733",-38.005155,...,NaN,http://www.properati.com.ar/197ur_venta_depart...,"Hall de entrada, Comedor con pisos graníticos ...",2 ambientes lateral soleado A/C,https://thumbs4.properati.com/1/x0ZXSiiHG5qMKR...,Buenos Aires Costa Atlántica,Mar del Plata,,None,POINT (-57.54447 -38.00516)


Se realiza un filtrado para saber cuantos datos no poseen ciudad

In [89]:
geo_data_filtrado = geo_data.loc[geo_data.Ciudad == "",:]
geo_data_filtrado.shape[0]

4780

Se busca observar cuantos datos pertenencen a Capital Federal, para saber su ciudad mediante el dataset de barrios, el cual posee la información de geolocalización de los barrios de Argentina.

In [90]:
geo_data_CABA = geo_data_filtrado.loc[geo_data.Provincia == "Capital Federal",["Unnamed: 0","lat","lon","geometry"]]
geo_data_CABA = geo_data_CABA.loc[geo_data_CABA.lat.notnull(),:]
geo_data_CABA

,Unnamed: 0,lat,lon,geometry
901,901,-34.610862,-58.411706,POINT (-58.41171 -34.61086)
1081,1081,-34.547904,-58.465996,POINT (-58.46600 -34.54790)
1498,1498,-34.614454,-58.385924,POINT (-58.38592 -34.61445)
1499,1499,-34.614454,-58.385924,POINT (-58.38592 -34.61445)
2844,2844,-34.602769,-58.368315,POINT (-58.36832 -34.60277)
...,...,...,...,...
120694,120694,-34.634536,-58.429898,POINT (-58.42990 -34.63454)
120814,120814,-34.631646,-58.434837,POINT (-58.43484 -34.63165)
120929,120929,-34.606977,-58.376367,POINT (-58.37637 -34.60698)
121057,121057,-34.636524,-58.429429,POINT (-58.42943 -34.63652)


In [91]:
data_barrios = "barrios.csv"
df_barrios = pd.read_csv(data_barrios,encoding="latin1")

import shapely.wkt
from shapely.geometry import Point
from shapely.geometry.polygon import Polygon

df_barrios["WKT"] = df_barrios["WKT"].apply(shapely.wkt.loads) 
geo_barrios = gpd.GeoDataFrame(df_barrios, geometry='WKT')
geo_barrios.head(2)

,WKT,BARRIO,COMUNA,PERIMETRO,AREA
0,"POLYGON ((-58.45282 -34.59599, -58.45366 -34.5...",CHACARITA,15.0,7725.695228,3.118101e+06
1,"POLYGON ((-58.46558 -34.59656, -58.46562 -34.5...",PATERNAL,15.0,7087.513295,2.229829e+06


In [92]:
geo_data_CABA["BARRIO"] = geo_data_CABA["geometry"].apply(lambda x: geo_barrios.loc[geo_barrios.contains(x),"BARRIO"].values[0])
geo_data_CABA

,Unnamed: 0,lat,lon,geometry,BARRIO
901,901,-34.610862,-58.411706,POINT (-58.41171 -34.61086),BALVANERA
1081,1081,-34.547904,-58.465996,POINT (-58.46600 -34.54790),NUÑEZ
1498,1498,-34.614454,-58.385924,POINT (-58.38592 -34.61445),MONSERRAT
1499,1499,-34.614454,-58.385924,POINT (-58.38592 -34.61445),MONSERRAT
2844,2844,-34.602769,-58.368315,POINT (-58.36832 -34.60277),SAN NICOLAS
...,...,...,...,...,...
120694,120694,-34.634536,-58.429898,POINT (-58.42990 -34.63454),PARQUE CHACABUCO
120814,120814,-34.631646,-58.434837,POINT (-58.43484 -34.63165),PARQUE CHACABUCO
120929,120929,-34.606977,-58.376367,POINT (-58.37637 -34.60698),SAN NICOLAS
121057,121057,-34.636524,-58.429429,POINT (-58.42943 -34.63652),PARQUE CHACABUCO


De esta manera, se obtiene los barrios para Capital Federal. Luego, se unifica el dataset spliteado (data_split) con los barrios_CABA.

In [93]:
barrios_CABA = geo_data_CABA.loc[:,"BARRIO"]
data_barrio = pd.merge(data_split,barrios_CABA,how="left",left_index=True,right_index=True)
data_barrio.sample(5)

,Unnamed: 0,operation,property_type,place_name,place_with_parent_names,country_name,state_name,geonames_id,lat-lon,lat,...,properati_url,description,title,image_thumbnail,Provincia,Ciudad,Barrio/Zona/Country,Country/Barrio_Cerrado,geometry,BARRIO
110921,110921,sell,house,Pinamar,|Argentina|Buenos Aires Costa Atlántica|Pinamar|,Argentina,Buenos Aires Costa Atlántica,3429971.0,"-37.0816717104,-56.8387328437",-37.081672,...,http://www.properati.com.ar/1c4y7_venta_casa_p...,AV DEL OLIMPO 952 E/ PENELOPE Y HELADES – PINA...,Casa con Piscina climatizada,https://thumbs4.properati.com/8/3QEUF65lgE7nXh...,Buenos Aires Costa Atlántica,Pinamar,,None,POINT (-56.83873 -37.08167),NaN
80135,80135,sell,apartment,Rosario,|Argentina|Santa Fe|Rosario|,Argentina,Santa Fe,3838574.0,"-32.9140305,-60.6745499",-32.914031,...,http://www.properati.com.ar/1awgs_venta_depart...,Sergio Villella Inmobiliaria presenta departa...,Departamento - Macrocentro,https://thumbs4.properati.com/6/BKtedNq9RYoyFp...,Santa Fe,Rosario,,None,POINT (-60.67455 -32.91403),NaN
48657,48657,sell,apartment,Monte Hermoso,|Argentina|Buenos Aires Costa Atlántica|Monte ...,Argentina,Buenos Aires Costa Atlántica,3843843.0,"-38.9886926221,-61.283021411",-38.988693,...,http://www.properati.com.ar/19ajw_venta_depart...,CODIGO: 1977-MHVD39 ubicado en: COSTANERA ESTE...,FRENTE AL MAR DE DOS DORMITORIOS Y COCHERA,https://thumbs4.properati.com/3/GFxZrH1f-4WU16...,Buenos Aires Costa Atlántica,Monte Hermoso,,None,POINT (-61.28302 -38.98869),NaN
49899,49899,sell,PH,Castelar,|Argentina|Bs.As. G.B.A. Zona Oeste|Morón|Cast...,Argentina,Bs.As. G.B.A. Zona Oeste,3435607.0,"-34.6538454,-58.6498624",-34.653845,...,http://www.properati.com.ar/19czl_venta_ph_cas...,Venta de Casa 3 AMBIENTES en CastelarTiene dos...,PH EN VENTA,https://thumbs4.properati.com/0/sAeCtF6sel4RN4...,Bs.As. G.B.A. Zona Oeste,Morón,Castelar,,POINT (-58.64986 -34.65385),NaN
87489,87489,sell,house,Bs.As. G.B.A. Zona Norte,|Argentina|Bs.As. G.B.A. Zona Norte|,Argentina,Bs.As. G.B.A. Zona Norte,3435907.0,NaN,NaN,...,http://www.properati.com.ar/1b5z5_venta_casa_b...,EL LUGAR: CHACRAS DEL PARANA está situado en Z...,Casa Zarate Chacras del Parana,https://thumbs4.properati.com/6/-teuZX2Mt9WKIl...,Bs.As. G.B.A. Zona Norte,,None,None,POINT (nan nan),NaN


Se crea una nueva columna "CiudadClean", donde se carga la información obtenida a los barrios de CABA. Y a los datos que no poseen barrio, se les asigna el cmapo "sin barrio".

In [94]:
data_barrio["CiudadClean"] = data_barrio["BARRIO"]

data_barrio.loc[np.logical_and(data_barrio.Provincia=="Capital Federal",
                               data_barrio.Ciudad==""),"CiudadClean"] =  data_barrio["CiudadClean"].fillna("sin barrio")

data_barrio.loc[np.logical_and(data_barrio.Provincia=="Capital Federal",
                               data_barrio.Ciudad!=""),"CiudadClean"] = data_barrio.loc[np.logical_and(data_barrio.Provincia=="Capital Federal",
                                                                                                       data_barrio.Ciudad!=""),"Ciudad"]

Luego se procede a la depuración de la columna "BarriosClean" y los barrios de CABA en la columna "CiudadClean".

In [95]:
barrios_caba = data_barrio.loc[data_barrio.Provincia=="Capital Federal","CiudadClean"]
barrios_caba = barrios_caba.apply(lambda x: x if x.isupper()== True else x.swapcase())
barrios_caba = barrios_caba.apply(lambda x: x if x.islower()== True else x.capitalize())
print(len(barrios_caba))
barrios_caba = barrios_caba.drop_duplicates()
print(len(barrios_caba))
barrios_caba = barrios_caba.sort_values()
dic = {"Constitucion":"Constitución","San nicolas":"San Nicolás",
       "Villa gral. mitre": "Villa general mitre","Villa pueyrredon":"Villa Pueyrredón","Villa devoto": "Villa Devoto",
      "Parque avellaneda":'Parque Avellaneda','Parque chas':'Parque Chas','Villa real':'Villa Real','Villa luro':'Villa Luro',
      'Villa crespo':'Villa Crespo'}
barrios_caba = barrios_caba.replace(dic)
barrios_caba = barrios_caba.drop_duplicates()
print(len(barrios_caba))

32316
63
61


In [96]:
data_barrio.loc[data_barrio.Provincia=="Capital Federal",
          "CiudadClean"] = data_barrio.loc[data_barrio.Provincia=="Capital Federal",
                                           "CiudadClean"].apply(lambda x: x if x.isupper()== True else x.swapcase())

data_barrio.loc[data_barrio.Provincia=="Capital Federal",
          "CiudadClean"] = data_barrio.loc[data_barrio.Provincia=="Capital Federal",
                                           "CiudadClean"].apply(lambda x: x if x.islower()== True else x.capitalize())

dic = {"Constitucion":"Constitución",
       "San nicolas":"San nicolás","Villa gral. mitre": "Villa general mitre","Villa pueyrredon":"Villa Pueyrredón"}

data_barrio.loc[data_barrio.Provincia=="Capital Federal",
          "CiudadClean"] = data_barrio.loc[data_barrio.Provincia=="Capital Federal","CiudadClean"].replace(dic)

data_barrio.loc[data_barrio.Provincia=="Capital Federal","CiudadClean"].value_counts()

Palermo                 4088
Belgrano                2998
Caballito               2306
Villa urquiza           1629
Recoleta                1558
Flores                  1358
Villa crespo            1344
San telmo               1216
Almagro                 1171
Barrio norte            1140
Boedo                    892
Nuñez                    779
Sin barrio               695
Balvanera                671
Puerto madero            647
San cristobal            615
Monserrat                551
Saavedra                 542
Floresta                 521
Villa del parque         488
Parque chacabuco         443
Villa luro               435
Barracas                 433
Villa devoto             422
Mataderos                418
Liniers                  408
Colegiales               353
Parque patricios         322
Congreso                 304
San nicolás              263
Coghlan                  259
Retiro                   259
Chacarita                227
Boca                     223
Centro / micro

De esta manera, se obtiene el dato de cuantas ciudades de CABA fueron imputadas por su barrio, en función de su geolocalización.

Luego, se procede a rellenar los campos nulos de "CiudadClean" con los datos que sí posee Ciudad, originalmente.

In [97]:
data_barrio.loc[data_barrio.Ciudad!="","CiudadClean"] = data_barrio.loc[data_barrio.Ciudad!="","Ciudad"]
print(data_barrio.CiudadClean.isnull().sum())
data_barrio.head(2)

3483


,Unnamed: 0,operation,property_type,place_name,place_with_parent_names,country_name,state_name,geonames_id,lat-lon,lat,...,description,title,image_thumbnail,Provincia,Ciudad,Barrio/Zona/Country,Country/Barrio_Cerrado,geometry,BARRIO,CiudadClean
0,0,sell,PH,Mataderos,|Argentina|Capital Federal|Mataderos|,Argentina,Capital Federal,3430787.0,"-34.6618237,-58.5088387",-34.661824,...,"2 AMBIENTES TIPO CASA PLANTA BAJA POR PASILLO,...",2 AMB TIPO CASA SIN EXPENSAS EN PB,https://thumbs4.properati.com/8/BluUYiHJLhgIIK...,Capital Federal,Mataderos,,None,POINT (-58.50884 -34.66182),NaN,Mataderos
1,1,sell,apartment,La Plata,|Argentina|Bs.As. G.B.A. Zona Sur|La Plata|,Argentina,Bs.As. G.B.A. Zona Sur,3432039.0,"-34.9038831,-57.9643295",-34.903883,...,Venta de departamento en décimo piso al frente...,VENTA Depto 2 dorm. a estrenar 7 e/ 36 y 37 ...,https://thumbs4.properati.com/7/ikpVBu2ztHA7jv...,Bs.As. G.B.A. Zona Sur,La Plata,,None,POINT (-57.96433 -34.90388),NaN,La Plata


Pero, se observa que 449 datos de cordoba sin ciudad tienen geolocalización.

In [98]:
data_cordoba = data_barrio.loc[np.logical_and(data_barrio.Provincia=="Córdoba",data_barrio.Ciudad==""),:]
data_cordoba_congeo = data_cordoba.loc[data_cordoba.lat.notnull(),:]
print(data_cordoba_congeo.shape)
data_cordoba_congeo.head(2)

(449, 33)


,Unnamed: 0,operation,property_type,place_name,place_with_parent_names,country_name,state_name,geonames_id,lat-lon,lat,...,description,title,image_thumbnail,Provincia,Ciudad,Barrio/Zona/Country,Country/Barrio_Cerrado,geometry,BARRIO,CiudadClean
1011,1011,sell,apartment,Córdoba,|Argentina|Córdoba|,Argentina,Córdoba,3860255.0,"-31.3995242,-64.1797594",-31.399524,...,CODIGO: 1277-VD3-001 ubicado en: GENERAL PAZ 1...,VENDO AMPLIO DEPARTAMENTO DE 3 DORMITORIOS Y D...,https://thumbs4.properati.com/8/wA3fwGQCzlDp6u...,Córdoba,,None,None,POINT (-64.17976 -31.39952),NaN,NaN
1023,1023,sell,house,Córdoba,|Argentina|Córdoba|,Argentina,Córdoba,3860255.0,"-31.4017668,-64.1645966",-31.401767,...,CODIGO: 2242-MAR60 ubicado en: CHARCAS 1782 - ...,Vendo Casa y Departamento de CATEGORÍA en Bº P...,https://thumbs4.properati.com/7/-5OGuwc8lxHlBg...,Córdoba,,None,None,POINT (-64.16460 -31.40177),NaN,NaN


Se obtieve un dataset de departamentos de Argentina, mediante la página https://www.ign.gob.ar/NuestrasActividades/InformacionGeoespacial/CapasSIG, para imputar estos datos.
En primer instancia, se realiza un chequeo del mismo.

In [99]:
data_departamentos = "departamento.csv"
df_departamentos = pd.read_csv(data_departamentos,encoding="latin1")
partidos = df_departamentos.groupby("fna").count()
partidos.sample(2)

,gid,objeto,geom,gna,nam,in1,fdc,sag
fna,,,,,,,,
Departamento PuelÃ©n,1,1,1,1,1,1,1,1
Partido de Berazategui,1,1,1,1,1,1,1,1


In [100]:
df_departamentos.geom = df_departamentos.geom.apply(shapely.wkt.loads) 
geo_departamentos = gpd.GeoDataFrame(df_departamentos, geometry='geom')
geo_departamentos.sample(2)

,gid,objeto,geom,fna,gna,nam,in1,fdc,sag
525,521,Departamento,"MULTIPOLYGON (((-68.88726 -29.62173, -68.88286...",Departamento JÃ¡chal,Departamento,JÃ¡chal,70056,Direc. de Catastro,IGN
88,66,Departamento,"MULTIPOLYGON (((-66.53119 -28.53104, -66.53220...",Departamento Arauco,Departamento,Arauco,46007,IGN,IGN


In [101]:
prueba = data_cordoba_congeo.loc[1011,"geometry"]
capital = geo_departamentos.loc[117,"geom"]
capital.contains(prueba) #121206

True

In [102]:
a = []
for i in data_cordoba_congeo.index:
    try:
        data_cordoba_congeo.loc[i,"BarriosCBA"] = geo_departamentos.loc[geo_departamentos.contains(data_cordoba_congeo.loc[i,"geometry"]),"fna"].values[0]
    except:
        a.append(i)

D:\Users\Lean\anaconda3\envs\dhdsblend2021_geopandas\lib\site-packages\pandas\core\indexing.py:1599: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = infer_fill_value(value)
D:\Users\Lean\anaconda3\envs\dhdsblend2021_geopandas\lib\site-packages\pandas\core\indexing.py:1720: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)


In [103]:
barrios_CBA = data_cordoba_congeo.loc[:,"BarriosCBA"]

data_barrios_CBA_CABA = pd.merge(data_barrio,barrios_CBA,how="left",left_index=True,right_index=True)

print(data_barrios_CBA_CABA.loc[np.logical_and(data_barrios_CBA_CABA.Provincia=="Córdoba",
                                               data_barrios_CBA_CABA.Ciudad==""),"BarriosCBA"].value_counts())

print(data_barrios_CBA_CABA.loc[np.logical_and(data_barrios_CBA_CABA.Provincia=="Córdoba",
                                               data_barrios_CBA_CABA.Ciudad!=""),"Ciudad"].value_counts())

Departamento Capital                335
Departamento Punilla                 52
Departamento ColÃ³n                  32
Departamento Calamuchita             11
Departamento Tercero Arriba           8
Departamento Caleu Caleu              3
Departamento Santa MarÃ­a             3
Departamento General San MartÃ­n      2
Departamento Cruz del Eje             1
Departamento El Cuy                   1
Name: BarriosCBA, dtype: int64
Córdoba             6606
Punilla              713
Villa Carlos Paz     612
Villa Allende        211
Santa María          155
                    ... 
Los Pozos              1
Marull                 1
Santa Catalina         1
Bouwer                 1
San Lorenzo            1
Name: Ciudad, Length: 113, dtype: int64


In [104]:
ciudades = data_barrios_CBA_CABA.loc[np.logical_and(data_barrios_CBA_CABA.Provincia=="Córdoba",
                                    data_barrios_CBA_CABA.Ciudad!=""),"Ciudad"].value_counts()
ciudades.sort_index()[0:50]

Achiras                   8
Agua de Oro               6
Almafuerte                7
Alta Gracia              18
Anisacate                 7
Ascochinga                7
Balnearia                 3
Bell Ville                2
Bialet Massé              6
Bouwer                    1
Cabalango                 2
Calmayo                   1
Capilla del Monte         9
Casa Grande               1
Colonia Caroya           14
Colonia Tirolesa          5
Cosquín                  14
Cruz del Eje              4
Córdoba                6606
Despeñaderos              3
El Manzano                2
Embalse                   5
Falda del Carmen          2
General Cabrera           1
Hernando                  3
Huerta Grande             9
Jesús María              17
La Calera               125
La Cumbre                10
La Cumbrecita             2
La Falda                 10
La Granja                16
La Paz                    4
La Serranita              1
Laboulaye                 3
Laguna Larga        

In [105]:
data_barrios_CBA_CABA.loc[np.logical_and(data_barrios_CBA_CABA.Provincia=="Córdoba",
                                         data_barrios_CBA_CABA.BarriosCBA=="Departamento Capital"),
                          "CiudadClean"] = "Córdoba"

data_barrios_CBA_CABA.loc[np.logical_and(data_barrios_CBA_CABA.Provincia=="Córdoba",
                                         data_barrios_CBA_CABA.CiudadClean=="Córdoba"),"Ciudad"]

10        Córdoba
32        Córdoba
87        Córdoba
88        Córdoba
89        Córdoba
           ...   
120686    Córdoba
120690    Córdoba
120966    Córdoba
120971    Córdoba
120979    Córdoba
Name: Ciudad, Length: 6941, dtype: object

In [106]:
data_barrios_CBA_CABA.loc[np.logical_and(data_barrios_CBA_CABA.Provincia=="Córdoba",
                                         data_barrios_CBA_CABA.BarriosCBA=="Departamento Santa MarÃ­a"),
                          "CiudadClean"] = "Santa María"

data_barrios_CBA_CABA.loc[np.logical_and(data_barrios_CBA_CABA.Provincia=="Córdoba",
                                         data_barrios_CBA_CABA.BarriosCBA=="Departamento Cruz del Eje"),
                          "CiudadClean"] = "Cruz del Eje"

#el resto de los departamentos no me dicen ciudades y pueden ser cualquiera de las ciudades del departamento
# veo calamuchita
data_barrios_CBA_CABA.loc[np.logical_and(data_barrios_CBA_CABA.Provincia=="Córdoba",
                                         data_barrios_CBA_CABA.BarriosCBA=="Departamento Calamuchita"),
                          ["lat","lon","title","description"]]

,lat,lon,title,description
18677,-32.069252,-64.537753,SANTA ROSA DE CALAMUCHITA - VENTA CASA A,Casa en Venta en Santa Rosa
41557,-32.178908,-64.445842,CASA EN VENTA EN EMBALSINA VILLA DEL DIQUE,SE VENDEExcelente ubicación a metros del lago....
41558,-32.182339,-64.451756,Oportunidad en Bello Horizonte,"SE VENDE Ubicada en Bello horizonte, Villa del..."
51897,-32.179050,-64.442073,VENDO CASA EN EMBALSINA HERMOSA VISTA AL LAGO,"Hermosa propiedad, consta de un lote de 1297 m..."
52980,-32.186785,-64.442189,SE VENDE CASA EN EMBALSINA HERMOSA VISTA SOB...,"La casa consta de dos dormitorios , living, co..."
60189,-32.064728,-64.535856,"HERMOSA CASA DE CATEGORÍA, SANTA ROSA DE",Casa en Venta de 3 dorm. en Santa Rosa
114684,-31.888207,-64.724391,Casa de campo en Villa Berna,Hermosa casa construida en piedra con destacad...
114685,-31.909078,-64.547976,Ciudad Parque Los Reartes- Vendo 2 casas idea...,Fracción de 525m2 con 2 casas de 36m2 cada una...
115146,-32.138409,-64.495711,CALAMUCHITA - Cód. 1206 - Vlla del Parque - ma...,Ubicada en Villa del Parque a 200 mts de la ru...
115400,-31.888233,-64.549116,Casa A Estrenar En Loteo Punto Claro - Loteo C...,"c346-Propiedad a estrenar en Loteo ""Punto Clar..."


Luego de la depuración me quedaron 3839 datos sin ciudad y tengo aprox 100 más con departamento pero sin ciudad.

In [107]:
data_barrios_CBA_CABA.loc[data_barrios_CBA_CABA.CiudadClean=="Sin barrio",
                          "CiudadClean"] = data_barrios_CBA_CABA.loc[data_barrios_CBA_CABA.CiudadClean=="Sin barrio",
                                                                     "BARRIO"]
print(data_barrios_CBA_CABA.CiudadClean.isnull().sum())

3839


In [108]:
data_barrios_CBA_CABA.loc[np.logical_and(data_barrios_CBA_CABA.Ciudad=="",
                                         data_barrios_CBA_CABA.CiudadClean.isnull()),"Provincia"].value_counts()

Córdoba                         2309
Capital Federal                  695
Bs.As. G.B.A. Zona Norte         222
Mendoza                          130
Buenos Aires Interior            106
Tucumán                           77
Bs.As. G.B.A. Zona Oeste          65
Misiones                          44
Santa Fe                          33
Buenos Aires Costa Atlántica      27
San Luis                          24
Bs.As. G.B.A. Zona Sur            24
Salta                             21
Chubut                            17
Neuquén                           10
Río Negro                          8
San Juan                           7
Tierra Del Fuego                   6
Catamarca                          3
Santiago Del Estero                3
La Pampa                           3
Corrientes                         2
Jujuy                              1
Chaco                              1
La Rioja                           1
Name: Provincia, dtype: int64

Este es el desgloce por provincia de los 3839 vacíos. Podría ver los departamentos para las otras propiedades que no tienen ciudad y que no forman parte de cordoba ni caba

<a id="section_countries"></a> 
### Regex: Countries

[volver a TOC](#section_toc)

Mediante expresiones regulares, se busca poder obtener información del campo description sobre posibles countries, ya que se considera un parámetro importante.

In [109]:
data_barrios_CBA_CABA.description = data_barrios_CBA_CABA.description.apply(str)
patron = "country|barrio cerrado|barrio privado|Country|Barrio cerrado|Barrio privado"
regex = re.compile(patron)
data_barrios_CBA_CABA["es_country"] = data_barrios_CBA_CABA.description.apply(lambda x: regex.search(x))
mask = data_barrios_CBA_CABA["es_country"].notnull()
data_barrios_CBA_CABA.loc[mask,"es_country"] = "si"
data_barrios_CBA_CABA.loc[mask,"es_country"].shape[0]

1933

In [110]:
data_barrios_CBA_CABA["Country/Barrio_Cerrado"].value_counts()

                             39892
Barrio Los Alisos              278
Barrio La Alameda               62
BarrioPortezuelo                54
Barrio Los Lagos                39
Barrio El Golf                  32
Enyoi                           17
Barrio Los Castores             14
QBay Yacht                      13
Barrio La Isla                  10
Barrio Las Glorietas            10
Islas del Canal                  6
Barrio Barrancas del Lago        4
Barrio Los Sauces                4
Barrio Cabos del Lago            3
Barrio El Yacht                  1
Barrio Los Tilos                 1
Name: Country/Barrio_Cerrado, dtype: int64

In [111]:
data_barrios_CBA_CABA.loc[np.logical_and(data_barrios_CBA_CABA["Country/Barrio_Cerrado"].notnull(),
                                         data_barrios_CBA_CABA["Country/Barrio_Cerrado"]!=""),"es_country"] = "si"

data_barrios_CBA_CABA["es_country"].value_counts()

si    2468
Name: es_country, dtype: int64

In [112]:
data_barrios_CBA_CABA.loc[np.logical_and(data_barrios_CBA_CABA["property_type"]=="house",
                                         data_barrios_CBA_CABA["Barrio/Zona/Country"]=="Nordelta"),"es_country"] = "si"

data_barrios_CBA_CABA["es_country"].value_counts()

si    3658
Name: es_country, dtype: int64

De esta manera, se pudieron obtener 36658 valores que son countries.

In [113]:
data_barrios_CBA_CABA.loc[np.logical_and(np.logical_and(data_barrios_CBA_CABA["property_type"]=="house",
                                        data_barrios_CBA_CABA["Barrio/Zona/Country"]!=""),
                         data_barrios_CBA_CABA.es_country!="si"),"Barrio/Zona/Country"].value_counts()

Martínez                 561
La Plata                 546
Benavidez                533
Adrogué                  502
Lomas de Zamora          457
                        ... 
Haras del Sur II           1
Fátima                     1
coordenadas 34.255511      1
Materno                    1
Juramento                  1
Name: Barrio/Zona/Country, Length: 516, dtype: int64

Luego, se realizó un pequeño dataset de countries con el fin de obtener información para complementar al dataset original

In [114]:
countries = ["Los Pingüinos Country Club","Haras Santa Maria","Santa Barbara Barrio Cerrado",
             "Mayling Club de Campo","Estancias del Pilar","La Comarca","Barrio San Rafael",
             "Olivos Golf Club","Fincas de Iraola","Barrio Santa Teresa","Barrio El Moro",
             "Acacias Blancas","Fincas de Hudson","Barrio Cerrado Las Casuarinas","Santa María de los Olivos",
             "Loma Verde","La Lomada de Pilar","Solar del Bosque","Solares del Talar","Septiembre","Santa-Catalina",
             "Carmel Country Club","Haras del Pilar - Las Praderas 1 y 2","La horqueta de Echeverría",
             "El Talar de Pacheco","El Encuentro","Country San Jorge Village","Campos de Álvarez",
             "Abril Club de Campo","Barrio San Agustin","Mapuche Country Club","Los Troncos",
             "La Peregrina","Talar del Lago 1","Laguna del Sol","Pacheco Golf Club",
             "Highland Park Country Club","El Rocío","Fincas de Iraola II",
             "GreenVille Polo & Resort","Country Banco Provincia","El Nacional Club de Campo","Altos del Pilar",
             "Golfer's Country Club","Haras María Victoria","Barrancas de San Jose","Barrio San Matías",
             "Talar del lago 2","Echeverría del Lago","Barrio Santa Clara","Valle Claro","Country Saint Thomas",
             "Barbarita, Barrio Cerrado","Country Nuevo Quilmes","Pilar del Lago","Fincas del Lago",
             "Club de Campo Pueyrredón","Haras María Elena","Los Lagartos Country Club","Los Pilares - Barrio Privado",
             "Barrio San Isidro Labrador","Barrio Santa Guadalupe - Pilar del Este","Albanueva Barrio Cerrado",
             "Boca Ratón","Pilar Village","Boat Center Barrio Cerrado","Brickland","Barrio Privado El Recodo  S.A.",
             "Barrio Cerrado El Lucero","Haras del Pilar - El Establo","Santa Isabel","Las Golondrinas",
             "Haras San Pablo","San Alberto","Barrio Ceibos","La Cesarina","Prados del Oeste","Troncos del Talar",
             "Altos de Hudson II","Barrancas de Santa María","La Madrugada","Pinos de Anchorena","Fincas de San Vicente",
             "Los Angeles Village","Bahía del Sol","Altos del Golf","Haras del Sur I","La Angélica",
             "Martindale Country Club","El Canton Barrio Puerto","Los Horneros CC","San Isidro Chico",
             "Los Robles de Maschwitz","Rincon Del Arca","Galapagos Country Club","Barrio Araucarias",
             "San Francisco Club de Campo","San Diego Country Club","El Remanso","Country Maschwitz Privado","Campos de Roca","Benavidez Greens","Bermudas Country Club","Los Pinares","Country Club Las lajas","Alto Los Cardales","Barrio cerrado Santa Ana","Barrio Cerrado Fincas de Maschwitz","Pilar Green Park","Lomas del Río Luján","Haras del Pilar - La Caballeriza","Barrio Acacias","Rincón de Maschwitz","Barrio San Eduardo - Pilar del Este","San Jose","San Lucas Village","Terravista Barrio Privado","Barrio Privado Santa Rita","Los Sauces Country Club","Barrio Vistas","Terralagos","Village Golf & Tennis Country Club","Barrio Alamo Alto","Grand Bell","St. Patrick Country","Club de Campo La Martona","Club El Carmen - Sector casas","Altos de Matheu","Barrio Parque Almirante Irízar","Hindu Club","Lomas de Benavidez","Los Cardales","Barrio Los Jazmines - Pilar del Este","Barrio Cerrado Los Abedules","Barrio La Cuesta","Altos de Manzanares 1 y 2","Larena Country Club","Miraflores Country Club","Country Maschwitz Club","Los Talas","La Herradura","Rincón de la Costa","San Eliseo Country, Golf, Hotel & Spa","El Pato Country Club","Country Club Aranjuez","Santa Catalina","Barrio San Marco","Hacoaj Barrio Cerrado","El Zorzal","The Boating Club","La Arboleda de Maschwitz","Haras del Sur II","Saint Matthews","Virasoro Village","Altos de Hudson I","Barrio Cerrado","Barrio Cerrado San Andres","Pilar Golf Country Club","La Escondida de Manzanares","Complejo de Barrios Privados La Magdalena","El Canton Barrio Islas","Islas","Barrio Cerrado Los Troncos","Armenia Country Club","Barrio Privado El Rodal","Barrio Cerrado El Casco de Alvarez","Marina Del Sol (Sun's Marine)","Barrio Cerrado Lagos del Norte","Barrio San Alfonso - Pilar del Este","El Molino","El Canton Barrio Norte","Barrio San Benito","Barrio Privado Villa Olivos","Los Tres Coniles","La Esperanza Club de Campo","Golf Club Argentino","Country Farm Club","Princess","Barrio Privado El Resuello","La Colina Golf & Polo","La Agustina","Barrio Cerrado"+"La Escondida"]


In [115]:
data_barrios_CBA_CABA.loc[data_barrios_CBA_CABA["Barrio/Zona/Country"].isin(countries),"es_country"] = "si"
data_barrios_CBA_CABA = data_barrios_CBA_CABA.rename(columns={"Barrio/Zona/Country":"Barrio_Zona_Country"})

patron = "country|barrio cerrado|barrio privado|Country|Barrio cerrado|Barrio privado|Club de campo|Club de Campo|Barrio Cerrado|Barrio Privado"
regex = re.compile(patron)


In [116]:
data_barrios_CBA_CABA["Barrio_Zona_Country"] = data_barrios_CBA_CABA["Barrio_Zona_Country"].astype(str)
data_barrios_CBA_CABA["a"] = data_barrios_CBA_CABA.Barrio_Zona_Country.apply(lambda x: regex.search(x))

In [117]:
mask = data_barrios_CBA_CABA["a"].notnull()
data_barrios_CBA_CABA.loc[mask,"a"] = "si"
data_barrios_CBA_CABA.loc[np.logical_and(mask,data_barrios_CBA_CABA.es_country!="si"),"es_country"] = "si" 
data_barrios_CBA_CABA["es_country"].value_counts()

si    5957
Name: es_country, dtype: int64

In [118]:
data_barrios_CBA_CABA["es_country_clean"] = data_barrios_CBA_CABA["es_country"]
data_barrios_CBA_CABA.loc[data_barrios_CBA_CABA["es_country"]!="si","es_country_clean"] = "no"
data_barrios_CBA_CABA.loc[:,"es_country_clean"].value_counts()

no    115263
si      5957
Name: es_country_clean, dtype: int64

De esta manera, se obtiene que 115263 no poseen countries, mientras que 5957 sí.

<a id="section_amenities"></a>
    
### Regex: Amenities

[volver a TOC](#section_toc)

De manera análoga, se hace lo mismo para con los Amenities

In [119]:
patron = "amenities|\sSUM\s|\ssum\s|pileta"
regex_amenities = re.compile(patron)
data_barrios_CBA_CABA["amenities"] = data_barrios_CBA_CABA.description.apply(lambda x: regex_amenities.search(x))
data_barrios_CBA_CABA.head()

,Unnamed: 0,operation,property_type,place_name,place_with_parent_names,country_name,state_name,geonames_id,lat-lon,lat,...,Barrio_Zona_Country,Country/Barrio_Cerrado,geometry,BARRIO,CiudadClean,BarriosCBA,es_country,a,es_country_clean,amenities
0,0,sell,PH,Mataderos,|Argentina|Capital Federal|Mataderos|,Argentina,Capital Federal,3430787.0,"-34.6618237,-58.5088387",-34.661824,...,,None,POINT (-58.50884 -34.66182),NaN,Mataderos,NaN,None,None,no,None
1,1,sell,apartment,La Plata,|Argentina|Bs.As. G.B.A. Zona Sur|La Plata|,Argentina,Bs.As. G.B.A. Zona Sur,3432039.0,"-34.9038831,-57.9643295",-34.903883,...,,None,POINT (-57.96433 -34.90388),NaN,La Plata,NaN,None,None,no,None
2,2,sell,apartment,Mataderos,|Argentina|Capital Federal|Mataderos|,Argentina,Capital Federal,3430787.0,"-34.6522615,-58.5229825",-34.652262,...,,None,POINT (-58.52298 -34.65226),NaN,Mataderos,NaN,None,None,no,None
3,3,sell,PH,Liniers,|Argentina|Capital Federal|Liniers|,Argentina,Capital Federal,3431333.0,"-34.6477969,-58.5164244",-34.647797,...,,None,POINT (-58.51642 -34.64780),NaN,Liniers,NaN,None,None,no,None
4,4,sell,apartment,Centro,|Argentina|Buenos Aires Costa Atlántica|Mar de...,Argentina,Buenos Aires Costa Atlántica,3435548.0,"-38.0026256,-57.5494468",-38.002626,...,Centro,,POINT (-57.54945 -38.00263),NaN,Mar del Plata,NaN,None,None,no,None


In [120]:
data_barrios_CBA_CABA.loc[np.logical_and(data_barrios_CBA_CABA["amenities"].notnull(),
                                         data_barrios_CBA_CABA["property_type"]=="apartment"),"amenities"] = "si"

data_barrios_CBA_CABA.loc[data_barrios_CBA_CABA.amenities != "si","amenities"] = "no"
data_barrios_CBA_CABA.sample(5)

D:\Users\Lean\anaconda3\envs\dhdsblend2021_geopandas\lib\site-packages\geopandas\array.py:689: RuntimeWarning: All-NaN slice encountered
  np.nanmin(b[:, 0]),  # minx
D:\Users\Lean\anaconda3\envs\dhdsblend2021_geopandas\lib\site-packages\geopandas\array.py:690: RuntimeWarning: All-NaN slice encountered
  np.nanmin(b[:, 1]),  # miny
D:\Users\Lean\anaconda3\envs\dhdsblend2021_geopandas\lib\site-packages\geopandas\array.py:691: RuntimeWarning: All-NaN slice encountered
  np.nanmax(b[:, 2]),  # maxx
D:\Users\Lean\anaconda3\envs\dhdsblend2021_geopandas\lib\site-packages\geopandas\array.py:692: RuntimeWarning: All-NaN slice encountered
  np.nanmax(b[:, 3]),  # maxy
D:\Users\Lean\anaconda3\envs\dhdsblend2021_geopandas\lib\site-packages\geopandas\array.py:689: RuntimeWarning: All-NaN slice encountered
  np.nanmin(b[:, 0]),  # minx
D:\Users\Lean\anaconda3\envs\dhdsblend2021_geopandas\lib\site-packages\geopandas\array.py:690: RuntimeWarning: All-NaN slice encountered
  np.nanmin(b[:, 1]),  # min

,Unnamed: 0,operation,property_type,place_name,place_with_parent_names,country_name,state_name,geonames_id,lat-lon,lat,...,Barrio_Zona_Country,Country/Barrio_Cerrado,geometry,BARRIO,CiudadClean,BarriosCBA,es_country,a,es_country_clean,amenities
11644,11644,sell,house,San Isidro,|Argentina|Bs.As. G.B.A. Zona Norte|San Isidro|,Argentina,Bs.As. G.B.A. Zona Norte,3428983.0,NaN,NaN,...,,None,POINT (nan nan),NaN,San Isidro,NaN,None,None,no,no
74974,74974,sell,apartment,Santa María,|Argentina|Córdoba|Santa María|,Argentina,Córdoba,3836128.0,NaN,NaN,...,,None,POINT (nan nan),NaN,Santa María,NaN,None,None,no,no
74091,74091,sell,apartment,Mar del Plata,|Argentina|Buenos Aires Costa Atlántica|Mar de...,Argentina,Buenos Aires Costa Atlántica,3430863.0,NaN,NaN,...,,None,POINT (nan nan),NaN,Mar del Plata,NaN,None,None,no,no
75121,75121,sell,house,Vicente López,|Argentina|Bs.As. G.B.A. Zona Norte|Vicente Ló...,Argentina,Bs.As. G.B.A. Zona Norte,NaN,NaN,NaN,...,Vicente López,,POINT (nan nan),NaN,Vicente López,NaN,None,None,no,no
25098,25098,sell,apartment,Las Grutas,|Argentina|Río Negro|Las Grutas|,Argentina,Río Negro,3837980.0,NaN,NaN,...,,None,POINT (nan nan),NaN,Las Grutas,NaN,None,None,no,no


Con esto queda finalizado la parte de Geopandas.

<a id="section_surface"></a> 

### Surface

[volver a TOC](#section_toc)

Primero se crea una función calcule el porcentaje de datos que poseen al menos 1 campo (total y/o covered),
luego que me dé el porcentaje que poseen solamente 1 campo (total o covered), que me dé el porcentaje que no
posea ningún campo y por último que me dé el porcentaje que tengan ambos campos



In [121]:
def data_surface_porc(data_filtered1,data_filtered2,data):

    mask = np.logical_or(data_filtered1.notnull(),data_filtered2.notnull()) #mascara or de datos no nulos
    data_filter = data[mask]
    datos = data_filter.shape[0] #numero de filas 

    mask2 = np.logical_xor(data_filtered1.notnull(),data_filtered2.notnull()) #mascara or exclusiva de datos no nulos
    data_filter2 = data[mask2]
    datos2 = data_filter2.shape[0]

    mask3 = np.logical_not(np.logical_or(data_filtered1.notnull(),data_filtered2.notnull())) #mascara nor de datos no nulos
    data_filter3 = data[mask3]
    datos3 = data_filter3.shape[0]

    mask4 = np.logical_and(data_filtered1.notnull(),data_filtered2.notnull()) #mascara and de datos no nulos
    data_filter4 = data[mask4]
    datos4 = data_filter4.shape[0]

    total = data.shape[0]
    porc_datos1 = (datos/total)*100
    porc_datos2 = (datos2/total)*100
    porc_datos3 = (datos3/total)*100
    porc_datos4 = (datos4/total)*100

    print('Tengo '+ str(np.round(porc_datos1,2)) + ' % de los datos que tienen al menos un campo lleno (surface o covered)')
    print('Tengo '+ str(np.round(porc_datos4,2)) + ' % de los datos que tienen ambos campos llenos')
    print('Tengo '+ str(np.round(porc_datos2,2)) + ' % de los datos que tienen solamente 1 campo lleno (surface o covered)')
    print('Tengo '+ str(np.round(porc_datos3,2)) + ' % de los datos que tienen nulos ambos campos (surface o covered)')

In [122]:
data_surface_porc(data_filtered1 = data_barrios_CBA_CABA.surface_total_in_m2,
                  data_filtered2 = data_barrios_CBA_CABA.surface_covered_in_m2, data = data_barrios_CBA_CABA)

Tengo 89.8 % de los datos que tienen al menos un campo lleno (surface o covered)
Tengo 61.34 % de los datos que tienen ambos campos llenos
Tengo 28.46 % de los datos que tienen solamente 1 campo lleno (surface o covered)
Tengo 10.2 % de los datos que tienen nulos ambos campos (surface o covered)


A partir de estos porcentajes, se procede a hacer el cociente entre covered y total con el fin de obtener el porcentaje
de cubierto de los valores que poseen ambos campos (61.34%).

In [123]:
mask = np.logical_and(data_barrios_CBA_CABA.surface_total_in_m2.notnull(),
                      data_barrios_CBA_CABA.surface_covered_in_m2.notnull())
data_filter = data_barrios_CBA_CABA[mask]
data_filter = data_filter.loc[:, ["property_type","surface_total_in_m2" ,"surface_covered_in_m2"]]
data_filter['columna'] = data_filter["surface_covered_in_m2"] / data_filter["surface_total_in_m2"]
data_filter.head()

,property_type,surface_total_in_m2,surface_covered_in_m2,columna
0,PH,55.0,40.0,0.727273
2,apartment,55.0,55.0,1.000000
4,apartment,35.0,35.0,1.000000
6,PH,106.0,78.0,0.735849
7,apartment,45.0,40.0,0.888889


Debido a que pueden estar cargado mal los datos (covered > total) se realiza una limpieza de esos registros

In [124]:
mask = data_filter["columna"] <= 1
data_filter = data_filter[mask]
data_filter

,property_type,surface_total_in_m2,surface_covered_in_m2,columna
0,PH,55.0,40.0,0.727273
2,apartment,55.0,55.0,1.000000
4,apartment,35.0,35.0,1.000000
6,PH,106.0,78.0,0.735849
7,apartment,45.0,40.0,0.888889
...,...,...,...,...
121215,apartment,113.0,93.0,0.823009
121216,house,360.0,360.0,1.000000
121217,apartment,46.0,39.0,0.847826
121218,apartment,48.0,48.0,1.000000


Por último, hacemos un groupby para obtener la media en función del tipo de propiedad, con estos valores
de porcentaje de media, vamos a rellenar el 28.46% que sólo poseen un solo campo.

In [125]:
data_filter_groupby = data_filter.groupby(by=['property_type']).mean()
data_filter_groupby

,surface_total_in_m2,surface_covered_in_m2,columna
property_type,,,
PH,117.787523,87.233876,0.809238
apartment,94.118849,66.518412,0.886848
house,524.763333,216.690709,0.684696
store,518.979034,455.051048,0.937632


In [126]:
data_surface = data_barrios_CBA_CABA.loc[:,['property_type','surface_total_in_m2','surface_covered_in_m2']]
data_surface = data_surface.sort_values(by=['property_type'])
mean = data_filter_groupby.columna
data_surface['col_PH'] = np.where(data_surface['property_type']== 'PH', mean[0], 0)
data_surface['col_apart'] = np.where(data_surface['property_type']== 'apartment', mean[1], 0)
data_surface['col_house'] = np.where(data_surface['property_type']== 'house', mean[2], 0)
data_surface['col_store'] = np.where(data_surface['property_type']== 'store', mean[3],0)
data_surface['col'] = data_surface['col_PH'] + data_surface['col_apart'] + data_surface['col_house'] + data_surface['col_store'] 

data_surface

,property_type,surface_total_in_m2,surface_covered_in_m2,col_PH,col_apart,col_house,col_store,col
0,PH,55.0,40.0,0.809238,0.0,0.0,0.000000,0.809238
65735,PH,30.0,30.0,0.809238,0.0,0.0,0.000000,0.809238
65739,PH,NaN,135.0,0.809238,0.0,0.0,0.000000,0.809238
65740,PH,NaN,NaN,0.809238,0.0,0.0,0.000000,0.809238
104922,PH,130.0,85.0,0.809238,0.0,0.0,0.000000,0.809238
...,...,...,...,...,...,...,...,...
90507,store,30.0,30.0,0.000000,0.0,0.0,0.937632,0.937632
76677,store,NaN,38.0,0.000000,0.0,0.0,0.937632,0.937632
76682,store,60.0,60.0,0.000000,0.0,0.0,0.937632,0.937632
13755,store,350.0,350.0,0.000000,0.0,0.0,0.937632,0.937632


Se procede a visualizar los datos que poseen al menos un campo completo

In [127]:
masked = np.logical_xor(data_surface.surface_covered_in_m2.notnull(),data_surface.surface_total_in_m2.notnull())
data_surface[masked]

,property_type,surface_total_in_m2,surface_covered_in_m2,col_PH,col_apart,col_house,col_store,col
65739,PH,NaN,135.0,0.809238,0.0,0.0,0.000000,0.809238
38783,PH,NaN,60.0,0.809238,0.0,0.0,0.000000,0.809238
38775,PH,NaN,70.0,0.809238,0.0,0.0,0.000000,0.809238
104868,PH,120.0,NaN,0.809238,0.0,0.0,0.000000,0.809238
11514,PH,NaN,169.0,0.809238,0.0,0.0,0.000000,0.809238
...,...,...,...,...,...,...,...,...
38262,store,NaN,20.0,0.000000,0.0,0.0,0.937632,0.937632
101982,store,NaN,100.0,0.000000,0.0,0.0,0.937632,0.937632
101862,store,370.0,NaN,0.000000,0.0,0.0,0.937632,0.937632
101890,store,NaN,35.0,0.000000,0.0,0.0,0.937632,0.937632


Completo el valor faltante con las medias calculadas. En este caso, se calcula la parte de covered utilizando la siguiente ecuacion: 
##### covered = media * total. 

In [128]:
data_surface_complete = data_surface.loc[:,['property_type','surface_total_in_m2','surface_covered_in_m2','col']]
data_surface_complete.surface_covered_in_m2 = data_surface_complete.apply(lambda x: x.col*x.surface_total_in_m2 
                                                                          if pd.isnull(x.surface_covered_in_m2) 
                                                                          else x.surface_covered_in_m2, axis=1)
data_surface_complete[masked]

,property_type,surface_total_in_m2,surface_covered_in_m2,col
65739,PH,NaN,135.000000,0.809238
38783,PH,NaN,60.000000,0.809238
38775,PH,NaN,70.000000,0.809238
104868,PH,120.0,97.108586,0.809238
11514,PH,NaN,169.000000,0.809238
...,...,...,...,...
38262,store,NaN,20.000000,0.937632
101982,store,NaN,100.000000,0.937632
101862,store,370.0,346.923952,0.937632
101890,store,NaN,35.000000,0.937632


Se puede observar que la fila con índice 104868 fue completado el valor de covered con el total * media.
De manera análoga y usando la expresión inversa:
##### (total = covered/media) 
Se realiza la estimación de la columna total.

In [129]:
data_surface_complete.surface_total_in_m2 = data_surface_complete.apply(lambda x: x.surface_covered_in_m2/x.col
                                                                          if pd.isnull(x.surface_total_in_m2) 
                                                                          else x.surface_total_in_m2, axis=1)
data_surface_complete[masked]

,property_type,surface_total_in_m2,surface_covered_in_m2,col
65739,PH,166.823560,135.000000,0.809238
38783,PH,74.143805,60.000000,0.809238
38775,PH,86.501105,70.000000,0.809238
104868,PH,120.000000,97.108586,0.809238
11514,PH,208.838383,169.000000,0.809238
...,...,...,...,...
38262,store,21.330323,20.000000,0.937632
101982,store,106.651616,100.000000,0.937632
101862,store,370.000000,346.923952,0.937632
101890,store,37.328065,35.000000,0.937632


Como se mencionó antes, las columnas poseen errores de imputación (total<covered), es por esto, que se imputan esos valores de total mediante la expresión anterior.

In [130]:
data_surface_complete.surface_total_in_m2 = data_surface_complete.apply(lambda x: x.surface_covered_in_m2/x.col
                                                                          if x.surface_total_in_m2 < x.surface_covered_in_m2
                                                                          else x.surface_total_in_m2, axis=1)

In [131]:
data_surface_complete1 = data_surface_complete.rename(columns={"surface_total_in_m2":"m2 Totales","surface_covered_in_m2":"m2 cubiertos"})
data_surface_complete1 = data_surface_complete1.drop("property_type",axis=1)
data_surface_complete1

,m2 Totales,m2 cubiertos,col
0,55.000000,40.0,0.809238
65735,30.000000,30.0,0.809238
65739,166.823560,135.0,0.809238
65740,NaN,NaN,0.809238
104922,130.000000,85.0,0.809238
...,...,...,...
90507,30.000000,30.0,0.937632
76677,40.527614,38.0,0.937632
76682,60.000000,60.0,0.937632
13755,350.000000,350.0,0.937632


In [132]:
data_complete = pd.merge(data_barrios_CBA_CABA,data_surface_complete1,how="left",left_index=True,right_index=True)
data_complete.head()

,Unnamed: 0,operation,property_type,place_name,place_with_parent_names,country_name,state_name,geonames_id,lat-lon,lat,...,BARRIO,CiudadClean,BarriosCBA,es_country,a,es_country_clean,amenities,m2 Totales,m2 cubiertos,col
0,0,sell,PH,Mataderos,|Argentina|Capital Federal|Mataderos|,Argentina,Capital Federal,3430787.0,"-34.6618237,-58.5088387",-34.661824,...,NaN,Mataderos,NaN,None,None,no,no,55.0,40.0,0.809238
1,1,sell,apartment,La Plata,|Argentina|Bs.As. G.B.A. Zona Sur|La Plata|,Argentina,Bs.As. G.B.A. Zona Sur,3432039.0,"-34.9038831,-57.9643295",-34.903883,...,NaN,La Plata,NaN,None,None,no,no,NaN,NaN,0.886848
2,2,sell,apartment,Mataderos,|Argentina|Capital Federal|Mataderos|,Argentina,Capital Federal,3430787.0,"-34.6522615,-58.5229825",-34.652262,...,NaN,Mataderos,NaN,None,None,no,no,55.0,55.0,0.886848
3,3,sell,PH,Liniers,|Argentina|Capital Federal|Liniers|,Argentina,Capital Federal,3431333.0,"-34.6477969,-58.5164244",-34.647797,...,NaN,Liniers,NaN,None,None,no,no,NaN,NaN,0.809238
4,4,sell,apartment,Centro,|Argentina|Buenos Aires Costa Atlántica|Mar de...,Argentina,Buenos Aires Costa Atlántica,3435548.0,"-38.0026256,-57.5494468",-38.002626,...,NaN,Mar del Plata,NaN,None,None,no,no,35.0,35.0,0.886848


Luego, se realiza un chequeo del porcentaje de los datos nulos y sus descriptores estadísticos principales.

In [133]:
data_surface_porc(data_filtered1 = data_complete['m2 Totales'], 
             data_filtered2 = data_complete['m2 cubiertos'],
             data = data_complete)

Tengo 89.8 % de los datos que tienen al menos un campo lleno (surface o covered)
Tengo 89.8 % de los datos que tienen ambos campos llenos
Tengo 0.0 % de los datos que tienen solamente 1 campo lleno (surface o covered)
Tengo 10.2 % de los datos que tienen nulos ambos campos (surface o covered)


In [134]:
data_complete.loc[:,['surface_total_in_m2','surface_covered_in_m2']].describe()

,surface_total_in_m2,surface_covered_in_m2
count,81892.000000,101313.000000
mean,233.795328,133.050181
std,1782.222147,724.351479
min,0.000000,0.000000
25%,50.000000,45.000000
50%,84.000000,75.000000
75%,200.000000,150.000000
max,200000.000000,187000.000000


In [135]:
data_complete.loc[:,['m2 Totales','m2 cubiertos']].describe()

,m2 Totales,m2 cubiertos
count,108851.000000,108851.000000
mean,227.441767,135.450147
std,1718.840440,900.597200
min,0.000000,0.000000
25%,51.000000,45.000000
50%,87.000000,74.000000
75%,209.865742,150.000000
max,210859.072241,187000.000000


Se puede apreciar que las columnas aún no están del todo bien imputadas, ya que los valores mínimos y máximos carecen de sentido (outlier). Este punto, se soluciona en la sección de filtrado.

<a id="section_regex"></a> 
## Regex Rooms

[volver a TOC](#section_toc)

In [136]:
data_regex = data_complete.loc[:,['description','title']]
data_regex.title = data_regex.title.astype(str)
data_regex.head()

,description,title
0,"2 AMBIENTES TIPO CASA PLANTA BAJA POR PASILLO,...",2 AMB TIPO CASA SIN EXPENSAS EN PB
1,Venta de departamento en décimo piso al frente...,VENTA Depto 2 dorm. a estrenar 7 e/ 36 y 37 ...
2,2 AMBIENTES 3ER PISO LATERAL LIVING COMEDOR AM...,2 AMB 3ER PISO CON ASCENSOR APTO CREDITO
3,PH 3 ambientes con patio. Hay 3 deptos en lote...,PH 3 amb. cfte. reciclado
4,DEPARTAMENTO CON FANTÁSTICA ILUMINACIÓN NATURA...,DEPTO 2 AMB AL CONTRAFRENTE ZONA CENTRO/PLAZA ...


In [137]:
patron = "(?P<g1>\d ambientes)|(?P<g2>\d ambiente)|(?P<g3>\d amb)|(?P<g4>\d AMB)|(?P<g5>\d AMBIENTE)|(?P<g6>\d AMBIENTES)|(?P<g7>\d Ambiente)|(?P<g8>\d Ambientes)|(?P<g9>\d aMBIE)|(?P<g10>\d Amb)"
regex_room = re.compile(patron)
data_regex["room"] = data_regex.description.apply(lambda x: regex_room.search(x))
data_regex

,description,title,room
0,"2 AMBIENTES TIPO CASA PLANTA BAJA POR PASILLO,...",2 AMB TIPO CASA SIN EXPENSAS EN PB,"<re.Match object; span=(0, 5), match='2 AMB'>"
1,Venta de departamento en décimo piso al frente...,VENTA Depto 2 dorm. a estrenar 7 e/ 36 y 37 ...,None
2,2 AMBIENTES 3ER PISO LATERAL LIVING COMEDOR AM...,2 AMB 3ER PISO CON ASCENSOR APTO CREDITO,"<re.Match object; span=(0, 5), match='2 AMB'>"
3,PH 3 ambientes con patio. Hay 3 deptos en lote...,PH 3 amb. cfte. reciclado,"<re.Match object; span=(3, 14), match='3 ambie..."
4,DEPARTAMENTO CON FANTÁSTICA ILUMINACIÓN NATURA...,DEPTO 2 AMB AL CONTRAFRENTE ZONA CENTRO/PLAZA ...,None
...,...,...,...
121215,TORRE FORUM ALCORTA - MÁXIMA CATEGORÍA.Impecab...,Torre Forum Alcorta- Impecable 3 ambientes,None
121216,Excelente e impecable casa en Venta en Las Lom...,Ruca Inmuebles | Venta | Lomas de San Isidro |...,None
121217,VENTA DEPARTAMENTO AMBIENTE DIVISIBLE A ESTREN...,VENTA DEPARTAMENTO AMBIENTE DIVISIBLE A ESTREN...,None
121218,"2 Amb al contrafrente, luminoso. El departame...",2 amb. C/ dep. de servicio al contrafrente| Re...,"<re.Match object; span=(0, 5), match='2 Amb'>"


In [138]:
data_regex.room = data_regex.room.apply(lambda x: x if x is None else x.group())
data_regex.room

0               2 AMB
1                None
2               2 AMB
3         3 ambientes
4                None
             ...     
121215           None
121216           None
121217           None
121218          2 Amb
121219           None
Name: room, Length: 121220, dtype: object

In [139]:
data_prueba = pd.merge(data.rooms,data_regex.room,how="left",left_index=True,right_index=True)
data_prueba['col'] = data_prueba.apply(lambda x: x.rooms if pd.isnull(x.room) else x.room, axis = 1)
data_regex_null = data_prueba.isnull().sum()
data_regex_null = data_regex_null.apply(lambda x: (x/data_regex.shape[0])*100)
data_regex_null

rooms    60.905791
room     69.041412
col      45.312655
dtype: float64

Mediante la técnica de Regex, se obtiene información sobre el campo Room, y se logra bajar su porcentaje a un 45 % de datos nulos.

<a id="section_filtrado"></a> 

### Filtrado

[volver a TOC](#section_toc)
       

In [140]:
data_final = pd.merge(data_complete,data_prueba['col'],how = 'left',left_index=True,right_index=True)
data_final.head()

,Unnamed: 0,operation,property_type,place_name,place_with_parent_names,country_name,state_name,geonames_id,lat-lon,lat,...,CiudadClean,BarriosCBA,es_country,a,es_country_clean,amenities,m2 Totales,m2 cubiertos,col_x,col_y
0,0,sell,PH,Mataderos,|Argentina|Capital Federal|Mataderos|,Argentina,Capital Federal,3430787.0,"-34.6618237,-58.5088387",-34.661824,...,Mataderos,NaN,None,None,no,no,55.0,40.0,0.809238,2 AMB
1,1,sell,apartment,La Plata,|Argentina|Bs.As. G.B.A. Zona Sur|La Plata|,Argentina,Bs.As. G.B.A. Zona Sur,3432039.0,"-34.9038831,-57.9643295",-34.903883,...,La Plata,NaN,None,None,no,no,NaN,NaN,0.886848,NaN
2,2,sell,apartment,Mataderos,|Argentina|Capital Federal|Mataderos|,Argentina,Capital Federal,3430787.0,"-34.6522615,-58.5229825",-34.652262,...,Mataderos,NaN,None,None,no,no,55.0,55.0,0.886848,2 AMB
3,3,sell,PH,Liniers,|Argentina|Capital Federal|Liniers|,Argentina,Capital Federal,3431333.0,"-34.6477969,-58.5164244",-34.647797,...,Liniers,NaN,None,None,no,no,NaN,NaN,0.809238,3 ambientes
4,4,sell,apartment,Centro,|Argentina|Buenos Aires Costa Atlántica|Mar de...,Argentina,Buenos Aires Costa Atlántica,3435548.0,"-38.0026256,-57.5494468",-38.002626,...,Mar del Plata,NaN,None,None,no,no,35.0,35.0,0.886848,NaN


In [141]:
data_final.columns

Index(['Unnamed: 0', 'operation', 'property_type', 'place_name',
       'place_with_parent_names', 'country_name', 'state_name', 'geonames_id',
       'lat-lon', 'lat', 'lon', 'price', 'currency',
       'price_aprox_local_currency', 'price_aprox_usd', 'surface_total_in_m2',
       'surface_covered_in_m2', 'price_usd_per_m2', 'price_per_m2', 'floor',
       'rooms', 'expenses', 'properati_url', 'description', 'title',
       'image_thumbnail', 'Provincia', 'Ciudad', 'Barrio_Zona_Country',
       'Country/Barrio_Cerrado', 'geometry', 'BARRIO', 'CiudadClean',
       'BarriosCBA', 'es_country', 'a', 'es_country_clean', 'amenities',
       'm2 Totales', 'm2 cubiertos', 'col_x', 'col_y'],
      dtype='object')

Lo primero que se hace es eliminar las columnas auxiliares utilizadas a modo de cálculo.

In [142]:
data_final = data_final.drop(['a','col_x','es_country','Ciudad','Unnamed: 0','BARRIO','BarriosCBA','Provincia'], axis=1)
data_final.head()

,operation,property_type,place_name,place_with_parent_names,country_name,state_name,geonames_id,lat-lon,lat,lon,...,image_thumbnail,Barrio_Zona_Country,Country/Barrio_Cerrado,geometry,CiudadClean,es_country_clean,amenities,m2 Totales,m2 cubiertos,col_y
0,sell,PH,Mataderos,|Argentina|Capital Federal|Mataderos|,Argentina,Capital Federal,3430787.0,"-34.6618237,-58.5088387",-34.661824,-58.508839,...,https://thumbs4.properati.com/8/BluUYiHJLhgIIK...,,None,POINT (-58.50884 -34.66182),Mataderos,no,no,55.0,40.0,2 AMB
1,sell,apartment,La Plata,|Argentina|Bs.As. G.B.A. Zona Sur|La Plata|,Argentina,Bs.As. G.B.A. Zona Sur,3432039.0,"-34.9038831,-57.9643295",-34.903883,-57.964330,...,https://thumbs4.properati.com/7/ikpVBu2ztHA7jv...,,None,POINT (-57.96433 -34.90388),La Plata,no,no,NaN,NaN,NaN
2,sell,apartment,Mataderos,|Argentina|Capital Federal|Mataderos|,Argentina,Capital Federal,3430787.0,"-34.6522615,-58.5229825",-34.652262,-58.522982,...,https://thumbs4.properati.com/5/SXKr34F_IwG3W_...,,None,POINT (-58.52298 -34.65226),Mataderos,no,no,55.0,55.0,2 AMB
3,sell,PH,Liniers,|Argentina|Capital Federal|Liniers|,Argentina,Capital Federal,3431333.0,"-34.6477969,-58.5164244",-34.647797,-58.516424,...,https://thumbs4.properati.com/3/DgIfX-85Mog5SP...,,None,POINT (-58.51642 -34.64780),Liniers,no,no,NaN,NaN,3 ambientes
4,sell,apartment,Centro,|Argentina|Buenos Aires Costa Atlántica|Mar de...,Argentina,Buenos Aires Costa Atlántica,3435548.0,"-38.0026256,-57.5494468",-38.002626,-57.549447,...,https://thumbs4.properati.com/5/xrRqlNcSI_vs-f...,Centro,,POINT (-57.54945 -38.00263),Mar del Plata,no,no,35.0,35.0,NaN


In [143]:
data_final = data_final.rename(columns = {'col_y':'rooms_regex'})
data_final.head()                              

,operation,property_type,place_name,place_with_parent_names,country_name,state_name,geonames_id,lat-lon,lat,lon,...,image_thumbnail,Barrio_Zona_Country,Country/Barrio_Cerrado,geometry,CiudadClean,es_country_clean,amenities,m2 Totales,m2 cubiertos,rooms_regex
0,sell,PH,Mataderos,|Argentina|Capital Federal|Mataderos|,Argentina,Capital Federal,3430787.0,"-34.6618237,-58.5088387",-34.661824,-58.508839,...,https://thumbs4.properati.com/8/BluUYiHJLhgIIK...,,None,POINT (-58.50884 -34.66182),Mataderos,no,no,55.0,40.0,2 AMB
1,sell,apartment,La Plata,|Argentina|Bs.As. G.B.A. Zona Sur|La Plata|,Argentina,Bs.As. G.B.A. Zona Sur,3432039.0,"-34.9038831,-57.9643295",-34.903883,-57.964330,...,https://thumbs4.properati.com/7/ikpVBu2ztHA7jv...,,None,POINT (-57.96433 -34.90388),La Plata,no,no,NaN,NaN,NaN
2,sell,apartment,Mataderos,|Argentina|Capital Federal|Mataderos|,Argentina,Capital Federal,3430787.0,"-34.6522615,-58.5229825",-34.652262,-58.522982,...,https://thumbs4.properati.com/5/SXKr34F_IwG3W_...,,None,POINT (-58.52298 -34.65226),Mataderos,no,no,55.0,55.0,2 AMB
3,sell,PH,Liniers,|Argentina|Capital Federal|Liniers|,Argentina,Capital Federal,3431333.0,"-34.6477969,-58.5164244",-34.647797,-58.516424,...,https://thumbs4.properati.com/3/DgIfX-85Mog5SP...,,None,POINT (-58.51642 -34.64780),Liniers,no,no,NaN,NaN,3 ambientes
4,sell,apartment,Centro,|Argentina|Buenos Aires Costa Atlántica|Mar de...,Argentina,Buenos Aires Costa Atlántica,3435548.0,"-38.0026256,-57.5494468",-38.002626,-57.549447,...,https://thumbs4.properati.com/5/xrRqlNcSI_vs-f...,Centro,,POINT (-57.54945 -38.00263),Mar del Plata,no,no,35.0,35.0,NaN


<a id="section_price"></a> 

### Criterio de filtrado: Price_aprox_usd

[volver a TOC](#section_toc)


Observando los descriptores estadísticos de la columna 'price_aprox_usd', puede observarse que posee outlier debido a los valores máximo y mínimos.

In [144]:
data_describe = data.describe()
data_describe.loc[:,['price_aprox_usd']]

,price_aprox_usd
count,1.008100e+05
mean,2.397006e+05
std,3.913239e+05
min,0.000000e+00
25%,8.973388e+04
50%,1.450000e+05
75%,2.650000e+05
max,4.654544e+07


En primer instancia, se procede a observar los valores no nulos de dicha columna ordenados.

In [145]:
mask = data.price_aprox_usd.notnull()
data_filter = data[mask]
by_price_aprox_usd = data_filter.sort_values('price_aprox_usd')
by_price_aprox_usd.loc[:,['price','price_aprox_usd','surface_total_in_m2',
                  'surface_covered_in_m2','surface_covered_in_m2','price_usd_per_m2',
                 'price_per_m2','floor','rooms','expenses','description','title','place_with_parent_names']]

,price,price_aprox_usd,surface_total_in_m2,surface_covered_in_m2,surface_covered_in_m2,price_usd_per_m2,price_per_m2,floor,rooms,expenses,description,title,place_with_parent_names
9761,0.0,0.00,NaN,NaN,NaN,NaN,NaN,2.0,3.0,NaN,Casa en Venta de 2 dorm. en Armenia,SE VENDE CASA EN EL BARRIO EL LIMONAR,|Argentina|Córdoba|
34666,80000.0,4666.62,55.0,55.0,55.0,84.847636,1454.545455,NaN,2.0,NaN,PLAN BAUEN PILAY 1 DORMITORIO SIN ADJUDICAR CO...,Departamento venta,|Argentina|Santa Fe|Rosario|
59875,84900.0,4952.45,60.0,NaN,NaN,82.540833,NaN,NaN,1.0,NaN,Exelente oportunidad. Venta de fondo de comerc...,Local venta,|Argentina|Buenos Aires Costa Atlántica|Mar de...
4399,5000.0,5000.00,NaN,52.0,52.0,NaN,96.153846,NaN,2.0,NaN,Corredor Responsable: Mauro Marvisi - CMCPSI 5...,"SIN ADELANTO, SIN INTERÉS Y CUOTAS FIJAS.",|Argentina|Bs.As. G.B.A. Zona Oeste|Tres de Fe...
52421,5000.0,5000.00,NaN,77.0,77.0,NaN,64.935065,NaN,3.0,NaN,Corredor Responsable: Mauro Marvisi - CMCPSI 5...,"MINIMO ADELANTO, SIN INTERÉS Y CUOTAS FIJAS.",|Argentina|Bs.As. G.B.A. Zona Oeste|Tres de Fe...
...,...,...,...,...,...,...,...,...,...,...,...,...,...
91289,16000000.0,16000000.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"En Bv Artigas 876 , Parque Jardín, La Falda .A...",Casa - La Falda,|Argentina|Córdoba|Punilla|
91479,19500000.0,19500000.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,\tCECILIA INMUEBLES OFRECE:INCREIBLE PROPIEDAD...,Casa - Parque Siquiman,|Argentina|Córdoba|Punilla|
107390,650000000.0,22980378.29,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"Casa con Calefacción central, Mendoza, San Roq...",Casa - San Vicente,|Argentina|Mendoza|San Roque|
91468,25000000.0,25000000.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"Casa con Agua Corriente, Córdoba, Punilla, por...",Casa - Parque Siquiman,|Argentina|Córdoba|Punilla|


Si se utiliza el criterio de eliminación de outlier (distancia intercuartilica * 1.5) el dataset sufría una pérdida de información importante. Es por eso, que mediante un análisis exploratorio, se decidió definir límites inferiores y superiores a "mano" para limpiar el dataset.

In [146]:
mask2 = np.logical_and(by_price_aprox_usd['price_aprox_usd'] <= 14000000,by_price_aprox_usd['price_aprox_usd'] >= 30000)
by_price_aprox_usd2 = by_price_aprox_usd[mask2]
by_price_aprox_usd2.loc[:,['price','price_aprox_usd','surface_total_in_m2',
                  'surface_covered_in_m2','surface_covered_in_m2','price_usd_per_m2',
                 'price_per_m2','floor','rooms','expenses','description','title','place_with_parent_names']]

,price,price_aprox_usd,surface_total_in_m2,surface_covered_in_m2,surface_covered_in_m2,price_usd_per_m2,price_per_m2,floor,rooms,expenses,description,title,place_with_parent_names
73953,30000.0,30000.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,SOBRE LA CALLE SEGUI Nº680 (ADROGUE)•\tFONDO...,Vende:Fondo de Comercio,|Argentina|Bs.As. G.B.A. Zona Sur|Almirante Br...
35939,30000.0,30000.0,16.0,16.0,16.0,1875.000000,1875.000000,NaN,NaN,NaN,"Local en el centro de la ciudad en ""Galería To...",Local en el centro de la ciudad,|Argentina|Buenos Aires Costa Atlántica|Mar de...
117583,30000.0,30000.0,16.0,NaN,NaN,1875.000000,NaN,NaN,NaN,NaN,Oportunidad local en G...,Local - Microcentro,|Argentina|Capital Federal|
100062,30000.0,30000.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"Propiedad en venta en La Reja, Moreno-Totalmen...",Casa - La Reja,|Argentina|Bs.As. G.B.A. Zona Oeste|Moreno|La ...
30447,30000.0,30000.0,15.0,NaN,NaN,2000.000000,NaN,NaN,4.0,NaN,Habitacion prioncipal amplia2 Dormitorios1 Liv...,Departamento venta,|Argentina|Bs.As. G.B.A. Zona Oeste|La Matanza...
...,...,...,...,...,...,...,...,...,...,...,...,...,...
45689,11111111.0,11111111.0,304.0,149.0,149.0,36549.707237,74571.214765,NaN,NaN,NaN,CODIGO: 2062-670 ubicado en: URQUIZA 7900 - P...,VENTA CASA AL FRENTE. 3 DORMITORIOS. 2 BAÑOS. ...,|Argentina|Santa Fe|Rosario|
114131,12000000.0,12000000.0,150.0,85.0,85.0,80000.000000,141176.470588,NaN,NaN,NaN,"CODIGO: ubicado en: MONSEÑOR TERRERO , Ituzai...","MONSEÑOR TERRERO , Ituzaingó, Buenos Aires.",|Argentina|Bs.As. G.B.A. Zona Oeste|Ituzaingó|
84400,13000000.0,13000000.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,AL VALOR PUBLICADO DEBERÁ ADICIONÁRSELE EL IMP...,Playa vehicular en venta-alquiler- Zarate,|Argentina|Bs.As. G.B.A. Zona Norte|
34638,14000000.0,14000000.0,2079.0,NaN,NaN,6734.006734,NaN,NaN,NaN,NaN,Excelente zona. Cerca de la Quinat Presidencia...,"Casa 2079m² en Del Libertador, Av. 2200, Vicen...",|Argentina|Bs.As. G.B.A. Zona Norte|Vicente Ló...


Luego, se sigue filtrando por los valores no nulos de los campos 'm2 Totales', 'price_aprox_usd' y los valores distintos de 0 de 'm2 Totales' y 'm2 cubiertos'

In [147]:
mask = np.logical_and(data_final['m2 Totales'].notnull(),data_final['price_aprox_usd'].notnull())
data_final_filter = data_final[mask]
mask2 = np.logical_and(data_final_filter['m2 Totales']!=0,data_final_filter['m2 cubiertos']!=0)
data_final_filter = data_final_filter[mask2]
data_final_filter.shape[0]

92923

In [148]:
mask2 = np.logical_and(data_final_filter['price_aprox_usd'] <= 14000000, data_final_filter['price_aprox_usd'] >= 30000)
data_final_filter2 = data_final_filter[mask2]
data_final_filter2.shape[0]

92432

<a id = "section_ciudad"></a>
    
### Filtrado Ciudad

[volver a TOC](#section_toc)

Además, se realiza un filtrado por las ciudades que no poseen información

In [149]:
mask3 = data_final_filter2.CiudadClean.notnull()
data_final_filter3 = data_final_filter2[mask3]
data_final_filter3.shape[0]

89859

In [150]:
data_final.columns

Index(['operation', 'property_type', 'place_name', 'place_with_parent_names',
       'country_name', 'state_name', 'geonames_id', 'lat-lon', 'lat', 'lon',
       'price', 'currency', 'price_aprox_local_currency', 'price_aprox_usd',
       'surface_total_in_m2', 'surface_covered_in_m2', 'price_usd_per_m2',
       'price_per_m2', 'floor', 'rooms', 'expenses', 'properati_url',
       'description', 'title', 'image_thumbnail', 'Barrio_Zona_Country',
       'Country/Barrio_Cerrado', 'geometry', 'CiudadClean', 'es_country_clean',
       'amenities', 'm2 Totales', 'm2 cubiertos', 'rooms_regex'],
      dtype='object')

<a id = "section_price_m2"></a>
         
### Estimación Precio x m2, en función de los datos limpios

[volver a TOC](#section_toc)

In [151]:
data_final_filter3['precio_por_m2_total'] = data_final_filter3.loc[:,"price_aprox_usd"]/data_final_filter3.loc[:,"m2 Totales"]
data_final_filter3['precio_por_m2_cubiertos'] = data_final_filter3.loc[:,"price_aprox_usd"]/data_final_filter3.loc[:,"m2 cubiertos"]

data_final_filter3.loc[:,['price_aprox_usd','m2 Totales','precio_por_m2_total','precio_por_m2_cubiertos']]

D:\Users\Lean\anaconda3\envs\dhdsblend2021_geopandas\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
D:\Users\Lean\anaconda3\envs\dhdsblend2021_geopandas\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,price_aprox_usd,m2 Totales,precio_por_m2_total,precio_por_m2_cubiertos
0,62000.0,55.0,1127.272727,1550.000000
2,72000.0,55.0,1309.090909,1309.090909
4,64000.0,35.0,1828.571429,1828.571429
6,130000.0,106.0,1226.415094,1666.666667
7,138000.0,45.0,3066.666667,3450.000000
...,...,...,...,...
121209,410000.0,157.0,2611.464968,2944.658184
121215,870000.0,113.0,7699.115044,9354.838710
121216,498000.0,360.0,1383.333333,1383.333333
121217,131500.0,46.0,2858.695652,3371.794872


In [152]:
data_final_filter4 = data_final_filter3.drop(['price_usd_per_m2','price_per_m2'],axis=1)

In [153]:
data.loc[:,['price_per_m2']].describe()

,price_per_m2
count,8.765800e+04
mean,6.912216e+03
std,2.837864e+04
min,1.510204e+00
25%,1.550000e+03
50%,2.213115e+03
75%,3.355549e+03
max,4.000000e+06


Observando los descriptores estadísticos de los precios por m2 originales, se observa que este campo posee outlier y el campo estimado también.

In [154]:
data_final_filter4.loc[:,['precio_por_m2_total','precio_por_m2_cubiertos']].describe()

,precio_por_m2_total,precio_por_m2_cubiertos
count,8.985900e+04,8.985900e+04
mean,2.537072e+03,3.145800e+03
std,1.512504e+04,1.831621e+04
min,1.000000e+00,1.127589e+00
25%,1.129748e+03,1.450000e+03
50%,1.730769e+03,2.000000e+03
75%,2.407407e+03,2.766990e+03
max,2.305805e+06,2.600000e+06


In [155]:
data_final_filter4.loc[:,['surface_total_in_m2','surface_covered_in_m2','precio_por_m2_total',
                          'precio_por_m2_cubiertos']].sort_values(by=['precio_por_m2_total','precio_por_m2_cubiertos'])

,surface_total_in_m2,surface_covered_in_m2,precio_por_m2_total,precio_por_m2_cubiertos
11534,49800.0,NaN,1.000000e+00,1.127589e+00
86675,107000.0,80.0,1.184571e+00,1.584364e+03
86698,107000.0,80.0,1.236986e+00,1.654468e+03
35151,52.0,49000.0,1.339322e+00,1.510204e+00
99248,47050.0,43.0,1.573442e+00,1.721638e+03
...,...,...,...,...
18451,NaN,1.0,7.531656e+05,1.100000e+06
349,NaN,1.0,1.096144e+06,1.236000e+06
58062,NaN,1.0,1.711179e+06,1.825000e+06
58009,NaN,1.0,2.156554e+06,2.300000e+06


Nuevamente, en este punto ocurre algo similar, utilizar el criterio del rango intercuartílico elimina mucha información del dataset, de manera que mediante un análisis exploratorio, se definir los límites de corte.

In [156]:
mask = (data_final_filter4.precio_por_m2_cubiertos > 500)&(data_final_filter4.precio_por_m2_cubiertos < 8400)
data_final_filter5 = data_final_filter4[mask]
data_final_filter5.loc[:,['surface_total_in_m2','surface_covered_in_m2','m2 Totales','m2 cubiertos','precio_por_m2_total',
                          'precio_por_m2_cubiertos']].sort_values(by=['precio_por_m2_cubiertos'])

,surface_total_in_m2,surface_covered_in_m2,m2 Totales,m2 cubiertos,precio_por_m2_total,precio_por_m2_cubiertos
82884,71.0,NaN,71.000000,62.966226,443.771831,500.392067
104713,116.0,116.0,116.000000,116.000000,500.401724,500.401724
45700,350.0,NaN,350.000000,239.643607,342.857143,500.743589
56746,350.0,NaN,350.000000,239.643607,342.857143,500.743589
52813,350.0,NaN,350.000000,239.643607,342.857143,500.743589
...,...,...,...,...,...,...
106044,70.0,NaN,70.000000,65.634261,7857.142857,8379.769793
69414,296.0,NaN,296.000000,262.507083,7432.432432,8380.726242
18170,422.0,316.0,422.000000,316.000000,6277.810427,8383.658228
62850,400.0,260.0,400.000000,260.000000,5450.000000,8384.615385


In [157]:
data_final_filter6 = data_final_filter5.drop(['surface_total_in_m2','surface_covered_in_m2','rooms'],axis=1)

In [158]:
data_null_clean = data_final_filter6.isnull().sum()
data_null_clean_porc = data_null_clean.apply(lambda x: (x/data_final_filter4.shape[0])*100)
data_null_clean_porc

operation                      0.000000
property_type                  0.000000
place_name                     0.024483
place_with_parent_names        0.000000
country_name                   0.000000
state_name                     0.000000
geonames_id                   16.936534
lat-lon                       38.925428
lat                           38.925428
lon                           38.925428
price                          0.000000
currency                       0.000000
price_aprox_local_currency     0.000000
price_aprox_usd                0.000000
floor                         89.049511
expenses                      82.476992
properati_url                  0.000000
description                    0.000000
title                          0.000000
image_thumbnail                1.783906
Barrio_Zona_Country            0.000000
Country/Barrio_Cerrado        59.766968
geometry                       0.000000
CiudadClean                    0.000000
es_country_clean               0.000000


Para finalizar la limpieza, se proccede a visualizar el porcentaje de los datos del nuevo dataset creado. La línea comentada, permite exportar este dataset en un archivo .csv

In [170]:
data_final_filter6['CiudadClean'] = data_final_filter6['CiudadClean'].apply(lambda x: x.lower())

In [171]:
len(np.unique(data_final_filter6.loc[:,['state_name','CiudadClean']]))

432

In [172]:
len(np.unique(data_final_filter6.loc[:,['CiudadClean']]))

405

In [174]:
group = data_final_filter6.loc[:,['state_name','CiudadClean']]
hola = group.pivot_table(group,aggfunc='count',index=['CiudadClean','state_name'])
hola.to_csv('prueba.csv')
hola

Empty DataFrame
Columns: []
Index: [(abasto, Capital Federal), (achiras, Córdoba), (agronomía, Capital Federal), (agua blanca, Catamarca), (agua de oro, Córdoba), (aguas verdes, Buenos Aires Costa Atlántica), (aldea brasilera, Entre Ríos), (almagro, Capital Federal), (almirante brown, Bs.As. G.B.A. Zona Sur), (alta gracia, Córdoba), (alvear, Santa Fe), (anisacate, Córdoba), (apóstoles, Misiones), (arequito, Santa Fe), (arroyo ceibal, Santa Fe), (arroyo seco, Santa Fe), (ascochinga, Córdoba), (avellaneda, Bs.As. G.B.A. Zona Sur), (azul, Buenos Aires Interior), (bahía blanca, Buenos Aires Interior), (balcarce, Buenos Aires Interior), (balvanera, Capital Federal), (baradero, Buenos Aires Interior), (barracas, Capital Federal), (barranqueras, Chaco), (barrio norte, Capital Federal), (belgrano, Capital Federal), (bella vista, Corrientes), (berazategui, Bs.As. G.B.A. Zona Sur), (berisso, Buenos Aires Interior), (bialet massé, Córdoba), (boca, Capital Federal), (boedo, Capital Federal), (brandsen, Buenos Aires Interior), (buena nueva, Mendoza), (cabalango, Córdoba), (caballito, Capital Federal), (cafayate, Salta), (caleta olivia, Santa Cruz), (calmayo, Córdoba), (camet, Buenos Aires Costa Atlántica), (campana, Buenos Aires Interior), (campo grande, Misiones), (campo quijano, Salta), (candelaria, Misiones), (capilla del monte, Córdoba), (capilla del rosario, Mendoza), (capitán bermúdez, Santa Fe), (capitán sarmiento, Buenos Aires Interior), (cariló, Buenos Aires Costa Atlántica), (carlos casares, Buenos Aires Interior), (carpintería, San Luis), (casa grande, Córdoba), (caseros, Entre Ríos), (catalinas, Capital Federal), (cañadón seco, Santa Cruz), (cañuelas, Bs.As. G.B.A. Zona Sur), (centenario, Neuquén), (centro / microcentro, Capital Federal), (cerrillos, Salta), (chacarita, Capital Federal), (chacras de coria, Mendoza), (chajarí, Entre Ríos), (chapadmalal, Buenos Aires Costa Atlántica), (chascomús, Buenos Aires Interior), (chivilcoy, Buenos Aires Interior), (choele choel, Río Negro), (cipolletti, Río Negro), (claromecó, Buenos Aires Costa Atlántica), (coghlan, Capital Federal), (colastiné, Santa Fe), (colegiales, Capital Federal), (colonia benitez, Chaco), (colonia caroya, Córdoba), (colonia tirolesa, Córdoba), (colón, Entre Ríos), (comodoro rivadavia, Chubut), (concepción del uruguay, Entre Ríos), (concordia, Entre Ríos), (confluencia, Neuquén), (congreso, Capital Federal), (constitución, Capital Federal), (corcovado, Chubut), (coronda, Santa Fe), (coronel dorrego, Buenos Aires Interior), (coronel moldes, Salta), (coronel suárez, Buenos Aires Interior), (corrientes, Corrientes), (cosquín, Córdoba), (costa azul, Buenos Aires Costa Atlántica), (costa del este, Buenos Aires Costa Atlántica), (costa esmeralda, Buenos Aires Costa Atlántica), (costa uruguay sur, Entre Ríos), (cruz del eje, Córdoba), (cushamen, Chubut), (córdoba, Córdoba), (despeñaderos, Córdoba), (dina huapi, Río Negro), (dunamar, Buenos Aires Costa Atlántica), (el calafate, Santa Cruz), ...]

[413 rows x 0 columns]

In [175]:
data_final_filter6.to_csv('data_clean.csv')